In [5]:
import pandas as pd

db = pd.read_csv("../lapacke_backend/total.csv", index_col=0)
db = db[~ db['arg_type'].map(lambda x: ('LAPACK' in x))]

wrapped = set(db['fct_name'].map(lambda x: x.replace("LAPACKE_", "")).unique())

import glob

prefix = "../../../lapack/SRC/"
file_list = glob.glob(prefix + "*.f")

file_names = set(map(lambda x: x.replace(prefix, "").replace(".f",""), file_list))

In [6]:
excluded_files = {f for f in file_names if not f in wrapped}
print("Files that should be not wrapped:")
print(excluded_files)

missing_files = {f for f in wrapped if not f in file_names}
print("Files missing:")
print(missing_files)

file_names = {f for f in file_names if f not in excluded_files and f not in missing_files}
file_list  = [prefix + f + ".f" for f in file_names] 

print(file_list[-1])

Files that should be not wrapped:
{'zhb2st_kernels', 'dlatps', 'claed7', 'slaebz', 'chetrs_aa_2stage', 'cgelq', 'zunbdb3', 'cla_gbrfsx_extended', 'drscl', 'zlamswlq', 'zlarfb', 'dlaed3', 'chetrs2', 'sgesc2', 'zla_gbrpvgrw', 'cgees', 'slaruv', 'cgeesx', 'cunbdb6', 'dlange', 'cheevr_2stage', 'dlaqz2', 'zunmr2', 'zgsvj1', 'zgsvj0', 'clarfgp', 'sgbrfsx', 'zhetri_3x', 'dlaein', 'zunm2r', 'slamtsqr', 'dlaqz1', 'zlaqgb', 'slaic1', 'clasr', 'stprfb', 'ssytrf_aa', 'ilaenv2stage', 'claqz0', 'ssyconvf', 'zlarscl2', 'zgghd3', 'dlasd2', 'slaneg', 'claqr4', 'zlanhe', 'zhetrs_aa_2stage', 'zhbevx_2stage', 'cgemqrt', 'dlaqtr', 'zhetrs_3', 'zla_herpvgrw', 'zla_geamv', 'dlaic1', 'sgeql2', 'zgeesx', 'dlarrv', 'claset', 'dlascl2', 'cunm2l', 'zsytri_3', 'ztrti2', 'dsygs2', 'zpbtf2', 'dgelst', 'zla_syamv', 'slange', 'dgesvdq', 'cheevd_2stage', 'slatsqr', 'dlarrc', 'zhesv_rook', 'dorm2l', 'zgejsv', 'clarz', 'zlargv', 'zlarcm', 'clarf', 'ssytri2', 'chesv_rook', 'claein', 'zheevd_2stage', 'dsytrf_aa_2stage', 's

#### Obtain, for each file, the list of arguments used.

In [8]:
def fetch_header(f):
    state = "searching"
    result = ""
    with open(f) as input:
        for line in input:
            if state == "searching":
                if line[0] != "*" and "subroutine" in line.lower():
                    state = "found"
                    result += line

            elif state == "found":
                if "==================================" in line:
                    return result
                elif line[0] == "*":
                    continue
                else:
                    result += line
            

def type_of(variable, declarations):
    # Declaration is a line of fortran declaring variable types.
    declaration = []
    for declaration in declarations:
        if any(map(lambda x: variable == x, declaration)):
            break

    result = ""

    types = ["character", "logic", "integer", "real", "double", "complex16", "complex"]
    for t in types:
        if any(map(lambda x: t in x, declaration)):
            result = t
            break
    
    # Lines declaring arrays include a parenthesis.
    if '(' in declaration:
        result = result + "s"
    
    return result


import re
# Format: list of type - var name, ordered in the header definition.
def extract_variables(header):
    # ( START_SEQUENCE ) END SEQUENCE
    start, end = header.find("(") + 1, header.find(")")
    var_names  = header[start:end].replace('\n','').replace('$','').replace(' ','').split(",")
    
    def extract_character_strings(input_string):
        # Use regular expression to find all words containing only characters
        character_strings = re.findall(r'\b[A-Za-z0-9_]+|\(', input_string.replace("*",""))
        return character_strings
    
    declarations = list(
        filter(
            lambda x: x != [], 
            map(extract_character_strings, header[end::].replace('\n     $','').split("\n"))
        )
    )

    return [(type_of(v, declarations), v) for v in var_names]


# Generate scalar variables.
space = "      "
output_name = "blas_pipe"
pipe_location = "../priv/pipe_file"
with open(output_name + ".f", "w") as output:
    output.write("""
      module blas_pipe
            use pipemodule
      contains
    """)
    for f in file_list:
        header = fetch_header(f).lower()
        blas_name = f.split("/")[-1][:-2].strip()

        output.write(header)
        output.write("      integer            pipe\n\n")
        
        output.write(f'      call pipe_open_write("{pipe_location}", pipe)\n')
        output.write(f'      call pipe_write_characters("{blas_name}", pipe)\n')
        for var_type, var_name in extract_variables(header):
            output.write(f"      call pipe_write_{var_type}({var_name}, pipe)\n")
        output.write(f"      call pipe_close(pipe)\n\n")

        output.write(f'      call pipe_open_read("{pipe_location}", pipe)\n')
        output.write('      call pipe_skip(pipe)\n')
        for var_type, var_name in extract_variables(header):
            output.write(f"      call pipe_read_{var_type}({var_name}, pipe)\n")
        output.write(f"      call pipe_close(pipe)\n")

        output.write("\n      END\n\n".lower())
    output.write("\n      end module blas_pipe")

